# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [5]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

In [3]:
ls

archive.zip
building-features.jpg
challenge-1.ipynb
challenge-2.ipynb
data-cleaning-results.png
nb-model-building.jpg
training.1600000.processed.noemoticon.csv


In [6]:
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maryramirez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/maryramirez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maryramirez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
# your code here
tweets = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-2', header = None)
tweets.columns = ['target','id','date','flag','user','text']
tweets_sample = tweets.sample(20000)
print(tweets_sample.shape)
tweets_sample.head()

(20000, 6)


,target,id,date,flag,user,text
1242156,4,1994166089,Mon Jun 01 11:31:01 PDT 2009,NO_QUERY,DSCMaeuschen,"Chips, chips, chips"
256233,0,1984677909,Sun May 31 15:35:05 PDT 2009,NO_QUERY,shutupmeg,(@khushee) No work today. Still feel like crap...
1258992,4,1998026148,Mon Jun 01 17:52:05 PDT 2009,NO_QUERY,ThisJane,@5zs i was laid off on friday = bad news. good...
38185,0,1573194603,Tue Apr 21 00:06:04 PDT 2009,NO_QUERY,JerriSternstein,sad 420/marathon monday is over
26283,0,1558676290,Sun Apr 19 09:04:59 PDT 2009,NO_QUERY,danileedlee,"@yourenotsorry aww babe, i'm so sorry"


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [10]:
# your code here
def clean_up(s):
    
   # Cleans up numbers, URLs, and special characters from a string.
    x = s.lower()
    clean_url = re.sub(r'http\S+', '', x)
    non_digits = re.sub(r'\d+',' ',clean_url) # Replace one or more digits by  ' '
    non_character = re.sub(r'\W+',' ',non_digits)
    non_spaces= re.findall('[a-z]+',non_character)
    #Args:
      #  test = ["@Ironhack's-#Q website 776-is http://ironhack.com [(2018)])"]
    #Returns:
    
    return " ".join(non_spaces)

In [18]:
#sentence = clean_up(text)
def tokenize(s):

  #Tokenize a string.

    tokens = nltk.word_tokenize(s) 
    
    return tokens
    

In [12]:
def stem_and_lemmatize(text):
 
    #Perform stemming and lemmatization on a list of words.
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    list = []
    
    #Args:
    for word in text:
        text_freq = lemmatizer.lemmatize(word)
        porter_stemmer = stemmer.stem(text_freq)
        list.append(porter_stemmer)


    #Returns:
    return list

In [13]:
def remove_stopwords(l):
 
    #Remove English stopwords from a list of strings.
    stop_words = list(stopwords.words('english')) 

    stop_text = [ i for i in l if not i in stop_words ]
       

    #Returns:
    return stop_text

In [20]:
clean_tweets = tweets_sample.copy()

In [21]:
clean_tweets['text_processed'] = clean_tweets['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)

In [22]:
print(clean_tweets.shape)
clean_tweets.head()

(20000, 7)


,target,id,date,flag,user,text,text_processed
1242156,4,1994166089,Mon Jun 01 11:31:01 PDT 2009,NO_QUERY,DSCMaeuschen,"Chips, chips, chips","[chip, chip, chip]"
256233,0,1984677909,Sun May 31 15:35:05 PDT 2009,NO_QUERY,shutupmeg,(@khushee) No work today. Still feel like crap...,"[khushe, work, today, still, feel, like, crap,..."
1258992,4,1998026148,Mon Jun 01 17:52:05 PDT 2009,NO_QUERY,ThisJane,@5zs i was laid off on friday = bad news. good...,"[z, wa, laid, friday, bad, news, good, news, t..."
38185,0,1573194603,Tue Apr 21 00:06:04 PDT 2009,NO_QUERY,JerriSternstein,sad 420/marathon monday is over,"[sad, marathon, monday]"
26283,0,1558676290,Sun Apr 19 09:04:59 PDT 2009,NO_QUERY,danileedlee,"@yourenotsorry aww babe, i'm so sorry","[yourenotsorri, aww, babe, sorri]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [23]:
# your code here
frequencies = nltk.FreqDist()

In [25]:
 # Filling the word frequencies in the review
for token in clean_tweets['text_processed']:
    for word in token:
        frequencies[word] += 1

   # return frequencies
    

In [28]:
corpus = list(frequencies.items())[:5000]
#corpus = list(frequencies.keys())[:5000]

In [29]:
corpus

[('chip', 19),
 ('khushe', 1),
 ('work', 1095),
 ('today', 853),
 ('still', 543),
 ('feel', 651),
 ('like', 1099),
 ('crap', 55),
 ('bed', 287),
 ('sleep', 396),
 ('onli', 342),
 ('bloodi', 22),
 ('bird', 39),
 ('outsid', 75),
 ('would', 352),
 ('shut', 30),
 ('z', 8),
 ('wa', 1276),
 ('laid', 5),
 ('friday', 110),
 ('bad', 343),
 ('news', 84),
 ('good', 1141),
 ('time', 813),
 ('complet', 34),
 ('day', 1324),
 ('challeng', 14),
 ('sad', 407),
 ('marathon', 15),
 ('monday', 113),
 ('yourenotsorri', 1),
 ('aww', 120),
 ('babe', 32),
 ('sorri', 339),
 ('much', 476),
 ('thing', 345),
 ('play', 224),
 ('american', 24),
 ('footbal', 12),
 ('earlyer', 1),
 ('badli', 16),
 ('hurt', 174),
 ('littl', 205),
 ('finger', 35),
 ('better', 236),
 ('abl', 53),
 ('write', 89),
 ('exam', 128),
 ('tomorrow', 437),
 ('x', 241),
 ('think', 626),
 ('felt', 30),
 ('heartwarm', 1),
 ('hear', 154),
 ('posit', 16),
 ('trait', 1),
 ('mood', 42),
 ('room', 85),
 ('cri', 95),
 ('realis', 14),
 ('shower', 61),
 ('

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [30]:
# your code here
nltk.NaiveBayesClassifier.train


<bound method NaiveBayesClassifier.train of <class 'nltk.classify.naivebayes.NaiveBayesClassifier'>>

In [32]:
#corpus = list(frequencies.items())[:5000]

def find_features(document):
    words = set(document)
    features = {}
    for i in corpus:
        features[i] = (i in words)
    return features

In [36]:
featureset = []

#for word in corpus:
#    x = find_features(corpus)
#    y = rating
#    concat = (x, y)
#    featureset.append(concat)

for word in range(len(clean_tweets.text)) :
    concat = ()
    x = find_features(clean_tweets['text'][word])
    y = True if clean_tweets['target'][word]==4 else False
    concat = (x, y)
    featureset.append(concat)

KeyError: 0

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [37]:
# your code here

featuresets = [(find_features(n), document) for (n, document) in range(corpus)]
train_set, test_set = featuresets[:1900], featuresets[1900:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

TypeError: 'list' object cannot be interpreted as an integer

In [34]:
classifier.show_most_informative_features()

NameError: name 'classifier' is not defined

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

print(nltk.classify.accuracy(classifier, test_set))


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here